In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np

In [ ]:

def load_best_fitness(file_path):
    try:
        with open(file_path, "rb") as f:
            results = pickle.load(f)
        generations = [data["generation"] for data in results]
        best_fitnesses = [data["best_fitness"] for data in results]
        best_solution = [data["best_solution"] for data in results]

        
        return generations, best_fitnesses, best_solution

    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return [], [], []
    except Exception as e:
        print(f"An error occurred while loading or processing the file: {e}")
        return [], [],[]


In [3]:
def plot_best_fitness(file_address, title, xlabel="Generation", ylabel="Best Fitness(Traveled Distance in µm)"):
    try:
        generations, best_fitnesses, best_solutions = load_best_fitness(file_address)
        plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.plot(best_fitnesses)
            
    except FileNotFoundError:
        print(f"Error: The file at {file_address} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
def plot_best_solution_vectors(file_address, title=""):
    generations, best_fitnesses, best_solutions = load_best_fitness(file_address)

    b_ind = max(best_fitnesses)
    b_index = best_fitnesses.index(b_ind)
    print("Best Fitness:", b_ind)
    print("Best Solution:", best_solutions[b_index])

    vectors = np.array(best_solutions[b_index]).reshape(-1, 2)

    starting_positions = [[i * 3, 0] for i in range(len(vectors))]

    scale_factor = 2

    plt.figure(figsize=(8, 4))
    for start, vec in zip(starting_positions, vectors):
        plt.quiver(
            start[0], start[1], vec[0] * scale_factor, vec[1] * scale_factor,
            angles='xy', scale_units='xy', scale=1, color='b', width=0.005
        )
        plt.scatter(start[0], start[1], color='red', zorder=5)

    plt.xlim(-2, len(vectors) * 3)
    plt.ylim(-2 * scale_factor, 2 * scale_factor)
    plt.xlabel("y-axis")
    plt.ylabel("z-axis")
    plt.axhline(0, color='black', linewidth=0.5, linestyle='--')
    plt.axvline(0, color='black', linewidth=0.5, linestyle='--')
    plt.grid(color='gray', linestyle=':', linewidth=0.5)
    plt.gca().set_aspect('equal', adjustable='box')

    ax = plt.gca()
    ax.set_xticks(ax.get_xticks())
    ax.set_yticks(ax.get_yticks())
    ax.set_xticklabels([f"{tick / scale_factor:.2f}" for tick in ax.get_xticks()])
    ax.set_yticklabels([f"{tick / scale_factor:.2f}" for tick in ax.get_yticks()])

    plt.title(title)
    plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_multiple_seeds(file_addresses, title, xlabel="Generation", ylabel="Best Fitness (Traveled Distance in µm)"):

    try:
        all_fitness_values = []

        for file_address in file_addresses:
            try:
                generations, best_fitnesses, best_solutions = load_best_fitness(file_address)
                all_fitness_values.append(best_fitnesses)
            except FileNotFoundError:
                print(f"Warning: The file at {file_address} was not found. Skipping this file.")
            except Exception as e:
                print(f"Warning: An error occurred while processing {file_address}. Skipping this file. Error: {e}")
        
        if not all_fitness_values:
            print("Error: No valid data found in the provided files.")
            return

        all_fitness_values = np.array(all_fitness_values)

        mean_fitness = np.mean(all_fitness_values, axis=0)
        std_fitness = np.std(all_fitness_values, axis=0)

        plt.figure(figsize=(10, 6))
        for seed_idx, fitness_values in enumerate(all_fitness_values):
            plt.plot(fitness_values, alpha=0.6, label=f'Seed {seed_idx + 1}')
        
        plt.plot(mean_fitness, color='black', label='Mean Fitness')
        plt.fill_between(
            range(len(mean_fitness)),
            mean_fitness - std_fitness,
            mean_fitness + std_fitness,
            color='gray',
            alpha=0.3,
            label='Std Dev'
        )

        plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.legend()
        plt.grid()
        plt.tight_layout()

        plt.show()

    except Exception as e:
        print(f"An error occurred: {e}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

def plot_multiple_seeds(file_addresses, title, negative = False, xlabel="Generation", ylabel="Best Fitness (Traveled Distance in mm)", image_name=  "./plot.jpg", draw_ref=False, change_tick=False):

    try:
        all_fitness_values = []

        for file_address in file_addresses:
            try:
                generations, best_fitnesses, best_solutions = load_best_fitness(file_address)
                best_fitnesses_mm = np.array(best_fitnesses) / 1000.0
                all_fitness_values.append(best_fitnesses_mm)
            except FileNotFoundError:
                print(f"Warning: The file at {file_address} was not found. Skipping this file.")
            except Exception as e:
                print(f"Warning: An error occurred while processing {file_address}. Skipping this file. Error: {e}")
        
        if not all_fitness_values:
            print("Error: No valid data found in the provided files.")
            return

        if (negative):
            all_fitness_values = [-fitness for fitness in all_fitness_values]

        max_fitness = np.max(all_fitness_values)
        print(f"The maximum fitness value across all seeds is: {max_fitness:.6f} mm")
        print("generation",len(best_fitnesses))
        all_fitness_values = np.array(all_fitness_values)

        mean_fitness = np.mean(all_fitness_values, axis=0)
        std_fitness = np.std(all_fitness_values, axis=0)
        min_fitness = np.min(all_fitness_values, axis=0)
        max_fitness = np.max(all_fitness_values, axis=0)

        generations = np.arange(1, len(mean_fitness) + 1)

        plt.figure(figsize=(10, 6))
        for seed_idx, fitness_values in enumerate(all_fitness_values):
            plt.plot(generations, fitness_values, alpha=0.6, label=f'Seed {seed_idx + 1}')
        
        plt.plot(generations, mean_fitness, color='black', label='Mean Fitness', linewidth=2)

        def sci_notation(x, pos):
            return f"{x:.1e}" 
        if change_tick == True:
            ax = plt.gca() 
            ax.yaxis.set_major_formatter(mticker.FuncFormatter(sci_notation)) 
        plt.fill_between(
            generations,
            min_fitness,
            max_fitness,
            color='lightblue',
            alpha=0.5,
            label='Min-Max Range'
        )
        if draw_ref == True:
            plt.axhline(y=2.6299, color='red', linestyle='--', linewidth=2, label=f'Original Design(2.62 mm)')

        plt.title(title, fontsize=16)
        plt.xlabel(xlabel, fontsize=12)
        plt.ylabel(ylabel, fontsize=12)
        plt.legend(fontsize=14)

        plt.grid()
        plt.tight_layout()
        pdf_file_path = image_name
        plt.savefig(pdf_file_path, format='pdf')
        plt.show()

    except Exception as e:
        print(f"An error occurred: {e}")


In [ ]:
cma_no_condition_file_paths = ["CMA-ES_no_condition/cmaes_results_24seed1.pkl","CMA-ES_no_condition/cmaes_results_24seed2.pkl","CMA-ES_no_condition/cmaes_results_24seed3.pkl","CMA-ES_no_condition/cmaes_results_24seed4.pkl","CMA-ES_no_condition/cmaes_results_24seed5.pkl"]
plot_multiple_seeds(
    file_addresses=cma_no_condition_file_paths,
    title="CMA-ES - Best Fitness Across Generations for Multiple Seeds",
    negative = False,
    draw_ref = True,
    image_name =  "./cma_es_no_condition.pdf"
)


In [ ]:
cma_no_condition_file_paths = ["CMA-ES_no_condition/cmaes_results_24seed1.pkl","CMA-ES_no_condition/cmaes_results_24seed2.pkl","CMA-ES_no_condition/cmaes_results_24seed3.pkl","CMA-ES_no_condition/cmaes_results_24seed4.pkl","CMA-ES_no_condition/cmaes_results_24seed5.pkl"] 
plot_multiple_seeds(
    file_addresses=cma_no_condition_file_paths,
    title="CMA-ES - Best Fitness Across Generations for Multiple Seeds",
    negative = False,
    draw_ref = True,
    image_name =  "./cma_es_no_condition.pdf"
)



In [ ]:
ga_file_paths = ["GA/ga_results_GA_Mar3_12_100_1-20250303-023931.pkl","GA/ga_results_GA_Mar3_12_100_2-20250303-023930.pkl", "GA/ga_results_GA_Mar3_12_100_3-20250303-023930.pkl","GA/ga_results_GA_Mar3_12_100_4-20250303-023930.pkl","GA/ga_results_GA_Mar3_12_100_5-20250303-040650.pkl"] 
plot_multiple_seeds(
    file_addresses=ga_file_paths,
    title="GA - Best Fitness Across Generations for Multiple Seeds",
    negative = True,
    draw_ref = True,
    image_name =  "./GA_multi_ref.pdf"
)

In [ ]:
cma_file_paths = ["CMA-ES/cmaes_results_10main_run_seed_f-20250228-040237.pkl", "CMA-ES/cmaes_results_20main_run_seed_f-20250228-040238.pkl",  "CMA-ES/cmaes_results_30main_run_seed_f-20250301-145142.pkl", "CMA-ES/cmaes_results_main_run_seed_f-20250228-040237.pkl","CMA-ES/cmaes_results_40main_run_seed_f-20250302-081007.pkl"]
plot_multiple_seeds(
    file_addresses=cma_file_paths,
    title="Best Fitness Across Generations for Multiple Seeds",
    draw_ref = True,
    image_name =  "./cma_es_with_reset_strategy.pdf"
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

num_cycles = 6 
t = np.linspace(0, num_cycles / 10, 500)

frq = 10
alpha = (t % (1 / frq)) * frq * np.deg2rad(45)  
magnitude = (t % (1 / frq) * frq) * 8  
theta = np.pi + alpha  

x = []
y = []

for th, mag in zip(theta, magnitude):
    rot_m = np.array([[np.cos(th), -np.sin(th)], [np.sin(th), np.cos(th)]])
    field_vector = rot_m @ np.array([[mag], [0]]) 
    
    x.append(field_vector[0, 0])
    y.append(field_vector[1, 0])

x = np.array(x)
y = np.array(y)

plt.figure(figsize=(10, 5))
plt.plot(t, x, label="y(t)", color='#24add6', linewidth=2)
plt.plot(t, y, label="z(t)", color='#d6274a', linewidth=2)

plt.xlabel("Time (s)", fontsize=16)
plt.ylabel("Magnetic Field Components (mT)", fontsize=14)
plt.title("Magnetic Field Components Over Time", fontsize=16)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.legend(fontsize=12)

plt.savefig("./field_plot.pdf", format='pdf', bbox_inches="tight")
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

def plot_mean_comparison(set1_addresses, set2_addresses, title, xlabel="Generation", ylabel="Best Fitness (Traveled Distance in mm)", 
                         image_name="./plot_comparison.pdf", draw_ref=False, change_tick=False, 
                         negate_set1=False, negate_set2=False, label_set1="Mean Fitness (Set 1)", label_set2="Mean Fitness (Set 2)"):
    def load_fitness_data(file_addresses):
        all_fitness_values = []
        for file_address in file_addresses:
            try:
                generations, best_fitnesses, best_solutions = load_best_fitness(file_address)
                best_fitnesses_mm = np.array(best_fitnesses) / 1000.0
                all_fitness_values.append(best_fitnesses_mm)
            except FileNotFoundError:
                print(f"Warning: The file at {file_address} was not found. Skipping.")
            except Exception as e:
                print(f"Warning: Error processing {file_address}. Skipping. Error: {e}")

        if not all_fitness_values:
            print("Error: No valid data found in the provided files.")
            return None
        return np.array(all_fitness_values)

    fitness_values_set1 = load_fitness_data(set1_addresses)
    fitness_values_set2 = load_fitness_data(set2_addresses)

    if fitness_values_set1 is None or fitness_values_set2 is None:
        print("Error: Could not load valid data for one or both sets.")
        return

    mean_fitness_set1 = np.mean(fitness_values_set1, axis=0)
    mean_fitness_set2 = np.mean(fitness_values_set2, axis=0)

    if negate_set1:
        mean_fitness_set1 *= -1
    if negate_set2:
        mean_fitness_set2 *= -1

    generations = np.arange(1, len(mean_fitness_set1) + 1)

    plt.figure(figsize=(10, 6))
    plt.plot(generations, mean_fitness_set1, color='#24add6', label=label_set1, linewidth=2)
    plt.plot(generations, mean_fitness_set2, color='#d6274a', label=label_set2, linewidth=2)

    if change_tick:
        ax = plt.gca()
        ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, pos: f"{x:.1e}"))

    if draw_ref:
        plt.axhline(y=2.6299, color='red', linestyle='--', linewidth=2, label=f'Original Design (2.62 mm)')

    # Customize plot
    plt.title(title,fontsize=16)
    plt.xlabel(xlabel,fontsize=12)
    plt.ylabel(ylabel,fontsize=12)
    plt.legend(fontsize=14)
    plt.grid()
    plt.tight_layout()
    
    # Save and show plot
    plt.savefig(image_name, format='pdf')
    plt.show()


In [ ]:
plot_mean_comparison(cma_no_condition_file_paths, ga_file_paths, "Comparison of Mean Fitness Between CMA-ES and GA", draw_ref=True,negate_set1=False, negate_set2=True, label_set1="Mean Fitness CMA-ES", label_set2="Mean Fitness GA", image_name="./plot_comparison_mean.pdf")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

def compute_magnetization(directions, B_r=0.0778):

    mu_0 = 4 * np.pi * 1e-7  
    M_magnitude_kA = (B_r / mu_0) / 1000
    
    directions = np.array(directions, dtype=float)

    magnetization_vectors = directions * M_magnitude_kA

    return magnetization_vectors

def plot_multiple_solution_vectors(file_addresses, name_list, title=""):
    default_vector = np.array([
        [0.707, 0.707], [0, 1], [-0.707, 0.707], 
        [-1, 0], [-0.707, -0.707], [0, -1], 
        [0.707, -0.707], [1, 0], [0.707, 0.707]
    ])
    default_name = "Original Design"

    all_vectors = []
    
    for file_address in file_addresses:
        generations, best_fitnesses, best_solutions = load_best_fitness(file_address)

        b_ind = max(best_fitnesses)
        b_index = best_fitnesses.index(b_ind)
        print(f"File: {file_address} | Best Fitness:", b_ind)

        vectors_b = np.array(best_solutions[b_index]).reshape(-1, 2)
        vectors = compute_magnetization(vectors_b)

        print("v_b", vectors_b, "vec", vectors)

        magnitudes = np.linalg.norm(vectors, axis=1)

        unit_vectors = np.zeros_like(vectors)
        nonzero_mask = magnitudes > 0 
        unit_vectors[nonzero_mask] = vectors[nonzero_mask] / magnitudes[nonzero_mask][:, np.newaxis] 

        all_vectors.append((unit_vectors, magnitudes))

    default_processed_vectors = compute_magnetization(default_vector)
    default_magnitudes = np.linalg.norm(default_processed_vectors, axis=1)

    default_unit_vectors = np.zeros_like(default_processed_vectors)
    nonzero_mask = default_magnitudes > 0
    default_unit_vectors[nonzero_mask] = default_processed_vectors[nonzero_mask] / default_magnitudes[nonzero_mask][:, np.newaxis]

    all_vectors.append((default_unit_vectors, default_magnitudes))
    name_list.append(default_name)
    flat_magnitudes = np.concatenate([mags for _, mags in all_vectors])
    norm = mcolors.Normalize(vmin=min(flat_magnitudes), vmax=max(flat_magnitudes))
    cmap = cm.viridis
    num_files = len(all_vectors)
    fig, axes = plt.subplots(num_files, 1, figsize=(8, 2 * num_files), sharex=True, sharey=True)

    if num_files == 1:
        axes = [axes]

    fixed_scale = 0.4

    for ax, (vectors, magnitudes), name in zip(axes, all_vectors, name_list):
        num_vectors = len(vectors)

        starting_positions = np.array([[i * 5, 0] for i in range(num_vectors)]) 

        print(f"Plotting {num_vectors} vectors for {name}")

        sm = cm.ScalarMappable(cmap=cm.Spectral_r, norm=norm)
        
        q = ax.quiver(
            starting_positions[:, 0], starting_positions[:, 1], 
            vectors[:, 0], vectors[:, 1],
            angles='xy', scale_units='xy', scale=fixed_scale, 
            color=sm.to_rgba(magnitudes),
            width=0.005
        )
        
        ax.scatter(starting_positions[:, 0], starting_positions[:, 1], 
                   color=sm.to_rgba(magnitudes), s=20, zorder=5)

        ax.set_ylim(-3, 3)
        ax.axhline(0, color='black', linewidth=0.5, linestyle='--')
        ax.axvline(0, color='black', linewidth=0.5, linestyle='--')
        ax.grid(color='gray', linestyle=':', linewidth=0.5)
        ax.set_aspect('equal') 
        ax.set_xlabel("x-axis", fontsize=12)
        ax.set_ylabel("y-axis", fontsize=12)
        ax.set_title(name, fontsize=14)
        ax.set_yticklabels([]) 

    fig.subplots_adjust(bottom=0.1, top=0.95, left=0.1, right=0.78, hspace=0.0)

    sm = cm.ScalarMappable(cmap=cm.Spectral_r, norm=norm)
    sm.set_array([])
    cbar_ax = fig.add_axes([0.83, 0.2, 0.02, 0.6])
    fig.colorbar(sm, cax=cbar_ax).set_label("Magnetization (KA/m)", fontsize=14)
    cbar_ax.tick_params(labelsize=12)
    if title:
        fig.suptitle(title, fontsize=12, fontweight="bold")

    pdf_file_path = "./multiple_vectors.pdf"
    plt.savefig(pdf_file_path, format='pdf', bbox_inches='tight')
    plt.show()

In [ ]:
plot_titles = ["seed 1", "seed 2", "seed 3", "seed 4","seed 5","Original Design"]
cma_file_paths = ["CMA-ES/cmaes_results_10main_run_seed_f-20250228-040237.pkl", "CMA-ES/cmaes_results_20main_run_seed_f-20250228-040238.pkl",  "CMA-ES/cmaes_results_30main_run_seed_f-20250301-145142.pkl", "CMA-ES/cmaes_results_main_run_seed_f-20250228-040237.pkl","CMA-ES/cmaes_results_40main_run_seed_f-20250302-081007.pkl"]
plot_multiple_solution_vectors( cma_file_paths
, plot_titles)
